In [1]:
#pragma cling add_include_path("/home/sz/Documents/workspace/third_party/minitorch")
#pragma cling add_library_path("/home/sz/Documents/workspace/third_party/minitorch/build/c10")
#pragma cling load("c10")

In [2]:
#include <c10/util/UniqueVoidPtr.h>

In [3]:
#include <c10/core/Device.h>

In [4]:
#include <c10/core/Allocator.h>

In [5]:
#include <vector>
#include <iostream>

## 1 create c10::DataPtr

In [6]:
std::cout << c10::DeviceType::CPU << std::endl;

cpu


In [7]:
c10:: Device device{c10::DeviceType::CPU};
//std::cout << device << std::endl;

In [8]:
std::vector<float> flatted_data_vector1d = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18};

In [9]:
float* float_ptr = flatted_data_vector1d.data();
std::cout << float_ptr << std::endl;

0x560e11679630


In [10]:
c10::DataPtr data_ptr{};
data_ptr = c10::DataPtr{flatted_data_vector1d.data(), device};

In [11]:
std::cout << data_ptr.device().type() << std::endl;
std::cout << data_ptr.get() << std::endl;

cpu
0x560e11679630


## 2 create storage

In [12]:
#include <c10/core/SymInt.h>
#include <c10/core/Storage.h>

In file included from input_line_18:1:
In file included from /home/sz/Documents/workspace/third_party/minitorch/c10/core/SymInt.h:3:
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:26:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    }
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:28:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:30:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:32:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/SymNodeImpl.h:34:5: error: control reaches end of non-void function [-Werror,-Wreturn-type]
    };
    ^
/home/sz/Documents/workspace/third_party/mi

In [13]:
std::cout << "hello" << std::endl;

hello


In [14]:
//c10::SymInt size_bytes{72};
std::size_t size_bytes =72;

In [15]:
// step2: create storage
// https://github.com/pytorch/pytorch/blob/master/c10/core/Storage.h
c10::Storage storage{/*use_byte_size*/c10::Storage::use_byte_size_t{}, /*size_bytes*/size_bytes, /*data_ptr*/std::move(data_ptr),nullptr, false};

In [16]:
std::cout << storage.nbytes() << std::endl;

72


###  DispatchKey and DispatchKeySet

In [26]:
#include <c10/core/DispatchKey.h>
#include <c10/core/DispatchKeySet.h>

In [27]:
std::cout << c10::DispatchKey::Undefined << std::endl;

Undefined


In [28]:
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::Undefined) << std::endl;

0


In [29]:
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::CPU) << std::endl;

44


In [30]:
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::CUDA) << std::endl;

45


In [31]:
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::Autocast) << std::endl;

30


In [32]:
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::AutocastCPU) << std::endl;
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::AutocastCUDA) << std::endl;
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::Autograd) << std::endl;
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::AutogradCPU) << std::endl;
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::AutogradCUDA) << std::endl;
std::cout << static_cast<std::uint16_t>(c10::DispatchKey::ADInplaceOrView) << std::endl;

27
30
118
104
105
22


In [33]:
c10::DispatchKeySet custom_dks{{c10::DispatchKey::AutocastCPU,c10::DispatchKey::AutogradCPU, c10::DispatchKey::CPU}};

In [36]:
std::cout << custom_dks << std::endl;

DispatchKeySet(CPU, AutogradCPU, AutocastCPU)


In [37]:
std::cout << custom_dks.raw_repr() << std::endl;

1236950597633


In [39]:
c10::DispatchKey custom_dk = custom_dks.highestPriorityTypeId();
std::cout << custom_dk << std::endl;

AutocastCPU


#### predefined DispatchKeySet

In [40]:
std::cout << c10::autograd_dispatch_keyset << std::endl;

DispatchKeySet(AutogradOther, AutogradNestedTensor)


In [41]:
std::cout << c10::autocast_dispatch_keyset << std::endl;

DispatchKeySet(AutocastCPU, AutocastXPU, AutocastHPU, AutocastCUDA)


In [42]:
std::cout << c10::default_included_set << std::endl;

DispatchKeySet(BackendSelect, ADInplaceOrView)


In [43]:
std::cout << c10::default_excluded_set << std::endl;

DispatchKeySet(AutocastCPU, AutocastXPU, AutocastHPU, AutocastCUDA)


In [58]:
std::cout << c10::python_ks << std::endl; 

DispatchKeySet(Python, PythonTLSSnapshot)


### DispatchKeySet operation

In [53]:
c10::DispatchKey custom_dk = custom_dks.highestPriorityTypeId();
std::cout << custom_dk << std::endl;

AutocastCPU


# caffe2::TypeMeta == tensor.dtype()

In [17]:
#include <c10/util/string_view.h>

In [18]:
c10::string_view a = "hello world";

In [19]:
std::cout << a << std::endl;
std::cout << a.size() << std::endl;

hello world
11


In [20]:
#include <c10/util/TypeIndex.h>

In [21]:
std::cout << c10::util::get_fully_qualified_type_name<float>() << std::endl;
std::cout << c10::util::get_fully_qualified_type_name<double>() << std::endl;

float
double


In [22]:
#include <c10/core/ScalarType.h>

In file included from input_line_28:1:
/home/sz/Documents/workspace/third_party/minitorch/c10/core/ScalarType.h:42:17: warning: enumeration values 'Undefined' and 'NumOptions' not handled in switch [-Wswitch]
        switch (t) {
                ^
/home/sz/Documents/workspace/third_party/minitorch/c10/core/ScalarType.h:46:9: warning: control may reach end of non-void function [-Wreturn-type]
        }
        ^


In [23]:
std::cout << c10::ScalarType::Float << std::endl;
std::cout << c10::NumScalarTypes << std::endl;
std::cout << c10::elementSize(c10::ScalarType::Float) << std::endl;

Float
8
4


In [24]:
#include <c10/util/typeid.h>

In [25]:
caffe2::TypeMeta float_dtype = caffe2::TypeMeta::Make<float>();
std::cout << float_dtype << std::endl;
std::cout << float_dtype.name() << std::endl;
std::cout << float_dtype.itemsize() << std::endl;

float
float
4
